# Classification de messages haineux/offensants
## Fine tune DistilBert sur le jeu de données Davidson nettoyé

Exécution > Modifier le type d'exécution > GPU

In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
# Import the necessary libraries:

import numpy as np
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

In [8]:
# vérifier si GPU
torch.cuda.is_available()

True

Importer les données de https://github.com/t-davidson/hate-speech-and-offensive-language/tree/master/data

In [9]:
# Load the dataset using pandas
df = pd.read_csv('./clean_davidson_labeled_data.csv', index_col=0)

In [10]:
# 24783 lignes
df.shape

(24783, 2)

In [11]:
# sélectionne seulement 50% des lignes
#df = df.sample(frac = 0.5)

In [12]:
df.sample(10)

,class,tweet
19048,1,my hobbies include being a sarcastic bitch an...
14455,1,Jackie a Old bamma ass bitch
21019,1,Some bitches are relentless. You can curve her...
19527,0,I don't fuck with bitch niggas I only fuck ni...
2915,1,he's a cheating wetback
8908,1,Eagles fans will always be annoying faggots
1056,1,;Oh this bitch Midy must be crazy
5797,1,not over no twitter shit. Plus youre a female...
11880,1,It aint nothing to cut that bitch off- K camp
110,0,""" you're fucking gay, blacklisted hoe"" Holding..."


In [13]:
# Create a train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(df['tweet'], df['class'], test_size=0.2, random_state=42)

In [14]:
# Instantiate the DistilBert tokenizer and encode the training and test texts
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)

In [15]:
# Convert the encoded texts and labels to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels.tolist())
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels.tolist())
)

Classification :
0 - hate speech
1 - offensive language
2 - neither

In [16]:
#Instantiate the DistilBertForSequenceClassification model
# Note that the num_labels parameter should be set to the number of classes in the dataset
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.

In [17]:
# Define the optimizer and the learning rate
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
#Define the training loop
def train(model, train_dataloader, optimizer):
    model.train()

    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

    return None


In [19]:
# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(3):
    train(model, train_dataloader, optimizer)

In [20]:
# Evaluate the model on the test set
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()

with torch.no_grad():
    test_loss, test_acc = 0, 0
    for batch in test_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        logits = outputs.logits

        test_loss += loss.item()
        test_acc += (logits.argmax(axis=1) == labels).sum().item()

    test_loss = test_loss/len(test_dataloader)
    print(test_loss)


0.3295558041705179


test loss de 0.28736890799576237 pour 50% du dataset Davidson.

test loss de 0.3295558041705179 (aussi eu 0.2787703068506333) pour 100% du dataset Davidson (environ 30 min).

In [21]:
model.save_pretrained("/content/save_model")

In [22]:
# test
# new_text = "RT @Scarlett_Iam: 'You'd rather live with trash,, than make it work with a good woman' #DiaryOfAMadBlackWoman"
# new_text = "you forgot to fucking call these bitches back!"
# new_text = "This is a new text that we want to classify."
# new_text = "Today is very sunny."
# new_text = "yeah bro plz stop calling me an idiot... are we in kindergarden you fucking faggot"
# new_text = "yeah bro plz stop calling me an idiot..."
new_text =  "are we at Ikea you fucking faggot"
# new_text = "it means that im following orders and being subordinate. sue me, ya dyke"
# new_text = "You'd better to die, go to hell"
# new_text = "Nice shirt!"
# new_text = "So come on"
#new_text = "dsfvxcfb2"

tokenized_text = tokenizer.encode_plus(
    new_text,
    add_special_tokens=True,
    return_attention_mask=True,
    return_tensors='pt'
)

# Si cette erreur :
# RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!
# (when checking argument for argument index in method wrapper_CUDA__index_select)
device =  torch.device('cpu')

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

with torch.no_grad():
    outputs = model(
        input_ids=tokenized_text['input_ids'],
        attention_mask=tokenized_text['attention_mask']
    )

# The probs variable will contain the predicted probabilities for each class.
# You can then take the class with the highest probability as the predicted label for the new text.
probs = torch.softmax(outputs.logits, dim=-1)
print(probs)

# Classification : 0 - hate speech, 1 - offensive language,  2 - neither

tensor([[0.4193, 0.5800, 0.0007]])


## Tests sur le jeu de données Vidgen

In [24]:
# Load Vidgen dataset using pandas
df = pd.read_csv('./vidgen_dataset.csv', index_col=0)

In [25]:
df.shape

(41144, 12)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41144 entries, 1 to 121100
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   acl.id          41144 non-null  object
 1   X1              41144 non-null  int64 
 2   text            41144 non-null  object
 3   label           41144 non-null  object
 4   type            41144 non-null  object
 5   target          41136 non-null  object
 6   level           41144 non-null  object
 7   split           41144 non-null  object
 8   round.base      41144 non-null  int64 
 9   annotator       41144 non-null  object
 10  round           41144 non-null  object
 11  acl.id.matched  30098 non-null  object
dtypes: int64(2), object(10)
memory usage: 4.1+ MB


In [27]:
df.head()

,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,1,NaN


In [28]:
df.drop(columns=["acl.id", "X1", "type", "target", "level", "split", "round.base", "annotator", "round", "acl.id.matched"], inplace=True)

In [29]:
df.reset_index(drop=True, inplace = True)

In [30]:
df.shape

(41144, 2)

In [31]:
df.tail()

,text,label
41139,Indian culture is so backwards,hate
41140,"Women don't want equality, they just want to b...",hate
41141,fuck covid,nothate
41142,This computer is garbage,nothate
41143,The only good Muslim is a dead one,hate


In [32]:
df.label.unique()

array(['hate', 'nothate'], dtype=object)

In [ ]:
df.label = pd.Categorical(df.label)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41144 entries, 0 to 41143
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    41144 non-null  object
 1   label   41144 non-null  object
dtypes: object(2)
memory usage: 643.0+ KB


In [34]:
df.sample(25)

,text,label
25290,Orcs are more clanny and secretive than the ot...,nothate
16832,as for the problem of some White women who glo...,hate
2581,"In Islam, Muslims cannot date outside of marri...",nothate
35269,I have a new name for them - turkroaches.,hate
1708,yids do not have integrity,hate
4993,resentment is not a quality with which women c...,nothate
34525,The Holohoax never happened did it . Mixing th...,hate
2820,your football coach called me and told me you ...,nothate
3003,"you are such a bad writer, you can not speak p...",hate
22171,he is probably sucking cock in a public bathro...,hate


In [43]:
df["hate speech proba"] = np.nan
df["offensive language proba"] = np.nan
df["neither proba"] = np.nan

In [44]:
df.head()

,text,label,hate speech proba,offensive language proba,neither proba
0,dalits are lowlives,hate,NaN,NaN,NaN
1,gay people are a burden to our society,hate,NaN,NaN,NaN
2,Arabs are not welcome,hate,NaN,NaN,NaN
3,I'm not saying we should actually eliminate he...,hate,NaN,NaN,NaN
4,bananas are for black people,hate,NaN,NaN,NaN


In [45]:
df["hate speech proba"] = df["hate speech proba"].astype(float)
df["offensive language proba"] = df["offensive language proba"].astype(float)
df["neither proba"] = df["neither proba"].astype(float)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41144 entries, 0 to 41143
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   text                      41144 non-null  object 
 1   label                     41144 non-null  object 
 2   hate speech proba         0 non-null      float64
 3   offensive language proba  0 non-null      float64
 4   neither proba             0 non-null      float64
dtypes: float64(3), object(2)
memory usage: 1.6+ MB


In [ ]:
for i in range(len(df)):
  new_text = df.loc[i].text

  tokenized_text = tokenizer.encode_plus(
      new_text,
      add_special_tokens=True,
      return_attention_mask=True,
      return_tensors='pt'
  )

  # Si cette erreur :
  # RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!
  # (when checking argument for argument index in method wrapper_CUDA__index_select)
  device =  torch.device('cpu')

  #device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

  model.to(device)

  with torch.no_grad():
      outputs = model(
          input_ids=tokenized_text['input_ids'],
          attention_mask=tokenized_text['attention_mask']
      )

  # The probs variable will contain the predicted probabilities for each class.
  # You can then take the class with the highest probability as the predicted label for the new text.
  probs = torch.softmax(outputs.logits, dim=-1).numpy()[0]



  df["hate speech proba"].loc[i] = probs[0]
  df["offensive language proba"].loc[i] = probs[1]
  df["neither proba"].loc[i] = probs[2]

  # Classification : 0 - hate speech, 1 - offensive language,  2 - neither

<ipython-input-58-7e549c8149f7>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hate speech proba"].loc[i] = probs[0]
<ipython-input-58-7e549c8149f7>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["offensive language proba"].loc[i] = probs[1]
<ipython-input-58-7e549c8149f7>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["neither proba"].loc[i] = probs[2]


In [62]:
df.sample(25)

,text,label,hate speech proba,offensive language proba,neither proba
25318,I wish Hitler had killed all of the Jews so th...,hate,NaN,NaN,NaN
4119,"They are shit, they both cheat on each other",nothate,0.002656,0.997160,0.000184
20539,"We should be suspicious of old people, they do...",hate,NaN,NaN,NaN
125,white chicks was a shit ass movie,nothate,0.002676,0.997194,0.000130
40428,i actively encourage the extinguishing of all ...,hate,NaN,NaN,NaN
20900,How did Hitler view Indians? I've never head t...,nothate,NaN,NaN,NaN
36103,Working mothers fuck up their children,hate,NaN,NaN,NaN
16042,We horny niggas matter too!!,nothate,0.227776,0.769631,0.002593
6527,I love how the majority of us will not tolerat...,nothate,0.597222,0.176524,0.226254
20169,The Hinch Jurneh 1. Set up an Instagram home a...,nothate,NaN,NaN,NaN


In [60]:
df.to_csv('./result_applied_Vidgen.csv')